In [ ]:
import sys
sys.path.append('../utils')
import generic
from folium.plugins import LocateControl


pygame 2.6.1 (SDL 2.28.4, Python 3.11.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
import sys
sys.path.append('../utils')
import generic
import pandas as pd
ovitraps_data = pd.read_csv('../data/banco_060924.csv',header=None, sep=';', encoding='utf-8')
ovitraps_data.drop(columns=[31], inplace=True)
ovitraps_data.columns = ["id_palheta",
                        "presenca_ovos",
                        "NUM_OVOS",
                        "IND_QTE_OVOS_DESIDRATADOS",
                        "IND_QTE_OVOS_ECLODIDOS",
                        "DT_INSTALACAO",
                        "DT_COLETA",
                        "NUM_SEMANA",
                        "ANO_REFERENCIA",
                        "NUM_ARMADILHA",
                        "ID_ARMADILHA",
                        "ID_ENDERECO_ARMADILHA",
                        "NUM_LOGRADOURO",
                        "NUM_SEQU_LOGRADOURO",
                        "NOM_LOGRADOURO",
                        "CEP",
                        "NUM_IMOVEL",
                        "COMPLEMENTO",
                        "IND_OFICIAL",
                        "QUADRA",
                        "VALOR_COORDENADA_X",
                        "VALOR_COORDENADA_Y",
                        "UF",
                        "COD_AREA_ABRANGENCIA",
                        "DES_MUNICIPIO",
                        "NUM_BAIRRO",
                        "NOM_BAIRRO",
                        "COD_DISTRITO_SANITARIO",
                        "TP_LOGRADOURO",
                        "DT_DESATIVACAO_ARMADILHA_ENDERECO",
                        "DT_CADASTRO_ARMADILHA_ENDERECO",]
ovitraps_with_coordinates = pd.read_csv('../data/processed/ovitraps_data_with_coordinates.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_7212\2154441945.py:2: DtypeWarning: Columns (8,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  ovitraps_data = pd.read_csv('../data/banco_060924.csv',header=None, sep=';')


In [ ]:
# keep first of the needed columns
traps_info = ovitraps_data.groupby('NUM_ARMADILHA').first()[['ID_ENDERECO_ARMADILHA','NOM_LOGRADOURO','NUM_IMOVEL','COMPLEMENTO','DT_DESATIVACAO_ARMADILHA_ENDERECO']]
coordinates = ovitraps_with_coordinates.groupby('narmad').first()[['latitude','longitude']]
coordinates.index = coordinates.index.map(lambda x:x - 900000)

traps_info = traps_info.merge(coordinates, left_on='NUM_ARMADILHA', right_on='narmad', how='left')
traps_info_filtered = traps_info[traps_info['DT_DESATIVACAO_ARMADILHA_ENDERECO'].isna()].drop(columns=['DT_DESATIVACAO_ARMADILHA_ENDERECO','ID_ENDERECO_ARMADILHA']).reset_index(drop=True)
traps_info_filtered['address'] = traps_info_filtered['NOM_LOGRADOURO'] + ', ' + traps_info_filtered['NUM_IMOVEL'].astype(str) + ' ' + traps_info_filtered['COMPLEMENTO'].fillna('') 
traps_info_filtered.drop(columns=['NOM_LOGRADOURO','NUM_IMOVEL','COMPLEMENTO'], inplace=True)
traps_info_filtered.dropna(axis=0, subset=['latitude','longitude'], inplace=True)


In [99]:
mymap = generic.create_map(traps_info_filtered[['latitude', 'longitude']].values)

mymap = generic.add_points_to_map(mymap,
                coordinates=traps_info_filtered[['latitude', 'longitude']].values,
                text=traps_info_filtered['address'].values,
                size=7)
LocateControl(auto_start=True).add_to(mymap)

mymap.save("../results/maps/ovitraps_map_gps.html")
